In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import pandas as pd
import seaborn as sns
import io #Used as buffer

In [2]:
nn_p=3
nn_w = 5
nn_tw = 1

# Functions

In [3]:
path= Path(f'../../data/cluster_results')
col_by_freq = True
col_palette = 'Set3'
pat = re.compile('.*/(exp\d)/(\w*)/nonn/freq([0-9\.]+)div(\w+)/(\w+)_\w+.csv')
decode_keys = ['experiment','function','freq','div','method']
# nn_pat = re.compile('.*/(exp\d)/(\w*)/nn/freq([0-9\.]+)\w+div([A-Za-z]+)/(\w+)_(\w+)_\w+.csv')
# nn_decode_keys = ['experiment','function','freq','div','method','replace_mech']

nn_pat = re.compile('.*/(exp\d)/(\w*)/nn/freq([0-9\.]+)nn_w(\d+)nn_p(\d+)\w+nn_tw(\d+)\w+div([A-Za-z]+)/(\w+)_(\w+)_\w+.csv')
nn_decode_keys = ['experiment','function','freq','nnw','nnp','nntw','div','method','replace_mech']#,


def get_nn_files(m): return list(path.glob(f'**/nn/**/*{m}.csv'))


def read_nn_csv(f,m):
    df = pd.read_csv(f)
    for k,v in zip(nn_decode_keys,nn_pat.search(str(f)).groups()): df[k] = v
    df['freq'] = df['freq'].astype(float)
    df['method'] = df['method'].str.replace('NNnorm_Worst', 'NN')
    df['method'] = df['method'] + '_' + df['div'] #'_' + df['replace_mech'] +
    
    df.drop(['replace_mech','div'], axis=1, inplace=True)
    df.rename({'0':m.upper(), m:m.upper()}, axis=1, inplace=True)
    return df

def get_data(m, normalize=False):
    nn_files = get_nn_files(m)
    data = pd.concat([read_nn_csv(f,m) for f in nn_files])
    data = data[data['nnw']==str(nn_w)]
    data = data[data['nntw']==str(nn_tw)]
    if normalize:
        data_norm = (data.groupby(['experiment','function','freq','method'])[m.upper()].mean().reset_index()
                         .groupby(['experiment','function'])[m.upper()].min().reset_index()
                         .rename({m.upper():m.upper()+'_norm'}, axis=1))
        data = data.merge(data_norm, 'left')
        data[m.upper()+'_norm'] = data[m.upper()] / data[m.upper()+'_norm']
    return data




# Table

In [4]:
df=get_data('nn_time', normalize=False)
df['function']=df['function'].str.title()
df.head()

,NN_TIME,experiment,function,freq,nnw,nnp,nntw,method
0,0.043077,exp1,Sphere,0.5,5,3,1,NN_HMu
1,0.042812,exp1,Sphere,0.5,5,3,1,NN_HMu
2,0.042622,exp1,Sphere,0.5,5,3,1,NN_HMu
3,0.043090,exp1,Sphere,0.5,5,3,1,NN_HMu
4,0.042758,exp1,Sphere,0.5,5,3,1,NN_HMu


In [5]:
table2=df.pivot_table(values='NN_TIME', index=['function','method'] ,columns=['experiment','freq'])
table2

experiment             exp1                                    exp2            \
freq                   0.5       1.0       4.0       10.0      0.5       1.0    
function   method                                                               
Rastrigin  NN_CwN  0.042169  0.021427  0.005297       NaN  0.042319  0.021574   
           NN_HMu  0.043429  0.022472       NaN       NaN  0.042782       NaN   
           NN_RI   0.043338  0.021543  0.005364  0.002148  0.043696  0.024794   
Rosenbrock NN_CwN  0.041581  0.020933  0.005304       NaN  0.041825  0.021913   
           NN_HMu  0.042705  0.022625       NaN       NaN  0.042614       NaN   
           NN_RI   0.043007  0.021520  0.005364  0.002182  0.043255  0.024937   
Sphere     NN_CwN  0.042203  0.020916  0.005316       NaN  0.043040  0.022041   
           NN_HMu  0.042916  0.021872       NaN       NaN  0.042782  0.022319   
           NN_RI   0.042936  0.021925  0.005428  0.002182  0.043140  0.021728   

experiment                                 exp3                                \
freq                   4.0       10.0      0.5       1.0       4.0       10.0   
function   method                                                               
Rastrigin  NN_CwN  0.005384       NaN  0.042894  0.021637  0.005525       NaN   
           NN_HMu       NaN       NaN  0.043356       NaN       NaN       NaN   
           NN_RI   0.005384  0.002203  0.043714  0.021963  0.005517  0.002194   
Rosenbrock NN_CwN  0.005320       NaN  0.042499  0.021830  0.005441       NaN   
           NN_HMu       NaN       NaN  0.043076       NaN       NaN       NaN   
           NN_RI   0.005355  0.002174  0.043415  0.021901  0.005524  0.002166   
Sphere     NN_CwN  0.005276       NaN  0.042293  0.021931  0.005442       NaN   
           NN_HMu       NaN       NaN  0.043851       NaN       NaN       NaN   
           NN_RI   0.005467  0.002171  0.044566  0.021574  0.005343  0.002251   

experiment             exp4                                
freq                   0.5       1.0       4.0       10.0  
function   method                                          
Rastrigin  NN_CwN  0.041396  0.020683  0.005211       NaN  
           NN_HMu  0.043924       NaN       NaN       NaN  
           NN_RI   0.042567  0.021383  0.005323  0.002170  
Rosenbrock NN_CwN  0.041454  0.020852  0.005158       NaN  
           NN_HMu  0.042642       NaN       NaN       NaN  
           NN_RI   0.043042  0.021770  0.005634  0.002198  
Sphere     NN_CwN  0.041310  0.020816  0.005251       NaN  
           NN_HMu  0.042360       NaN       NaN       NaN  
           NN_RI   0.043232  0.021363  0.005655  0.002190

In [6]:
print(table2.to_csv(float_format='%.3f'))

experiment,,exp1,exp1,exp1,exp1,exp2,exp2,exp2,exp2,exp3,exp3,exp3,exp3,exp4,exp4,exp4,exp4
freq,,0.5,1.0,4.0,10.0,0.5,1.0,4.0,10.0,0.5,1.0,4.0,10.0,0.5,1.0,4.0,10.0
function,method,,,,,,,,,,,,,,,,
Rastrigin,NN_CwN,0.042,0.021,0.005,,0.042,0.022,0.005,,0.043,0.022,0.006,,0.041,0.021,0.005,
Rastrigin,NN_HMu,0.043,0.022,,,0.043,,,,0.043,,,,0.044,,,
Rastrigin,NN_RI,0.043,0.022,0.005,0.002,0.044,0.025,0.005,0.002,0.044,0.022,0.006,0.002,0.043,0.021,0.005,0.002
Rosenbrock,NN_CwN,0.042,0.021,0.005,,0.042,0.022,0.005,,0.042,0.022,0.005,,0.041,0.021,0.005,
Rosenbrock,NN_HMu,0.043,0.023,,,0.043,,,,0.043,,,,0.043,,,
Rosenbrock,NN_RI,0.043,0.022,0.005,0.002,0.043,0.025,0.005,0.002,0.043,0.022,0.006,0.002,0.043,0.022,0.006,0.002
Sphere,NN_CwN,0.042,0.021,0.005,,0.043,0.022,0.005,,0.042,0.022,0.005,,0.041,0.021,0.005,
Sphere,NN_HMu,0.043,0.022,,,0.043,0.022,,,0.044,,,,0.042,,,
Sphere,NN_RI,0.043,0.022,0.005,0.002,0.043,0.022,0.005,0.002,0.045,0.022,0.005,0.002,0.043,0.021,0.006,0.002

